In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.feature_extraction import settings
from tsfresh.utilities.dataframe_functions import impute

In [3]:
# Navegar hasta la raíz del proyecto 
WORKSPACE = os.path.abspath(os.path.join(os.getcwd(), '../../'))

# Definir la ruta de la carpeta de datos
DATA_FOLDER = os.path.join(WORKSPACE, 'data')

print("Workspace:", WORKSPACE)
print("Data folder:", DATA_FOLDER)

Workspace: /home/jovyan/work/mlds6
Data folder: /home/jovyan/work/mlds6/data


In [4]:
# Cargar el dataset y eliminar filas con valores vacíos en 'PTPM_CON'
df = pd.read_csv(os.path.join(DATA_FOLDER, 'data_filtrado.csv'), usecols=['PTPM_CON', 'Fecha', 'CodigoEstacion'])

# Convertir la columna 'Fecha' a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Ordenar el DataFrame por 'CodigoEstacion' y 'Fecha'
df = df.sort_values(by=['CodigoEstacion', 'Fecha'])

In [6]:
# Aplicar la función de rolling por cada estación
df_rolled = []
for station, group in df.groupby('CodigoEstacion'):
    rolled_group = roll_time_series(
        group,
        column_id="CodigoEstacion",
        column_sort="Fecha",
        max_timeshift=6,
        min_timeshift=6,
    )
    df_rolled.append(rolled_group)

# Combinar los resultados de todos los grupos
df_rolled = pd.concat(df_rolled).reset_index(drop=True)

# Verificar el DataFrame resultante
print(df_rolled.head())

/usr/local/lib/python3.11/dist-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 158/158 [00:02<00:00, 67.08it/s]
/usr/local/lib/python3.11/dist-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 159/159 [00:02<00:00, 65.33it/s]
/usr/local/lib/python3.11/dist-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████| 158/158 [00:02<00:00, 69.50it/s]
/usr/local/lib/python3.11/dist-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  wa

       Fecha  CodigoEstacion  PTPM_CON                               id
0 2000-01-31        15070090       0.0  (15070090, 2000-02-06 00:00:00)
1 2000-02-01        15070090       0.0  (15070090, 2000-02-06 00:00:00)
2 2000-02-02        15070090       0.0  (15070090, 2000-02-06 00:00:00)
3 2000-02-03        15070090       0.0  (15070090, 2000-02-06 00:00:00)
4 2000-02-04        15070090       0.0  (15070090, 2000-02-06 00:00:00)


In [7]:
X = extract_features(df_rolled.drop("CodigoEstacion", axis=1), 
                     column_id="id", column_sort="Fecha", column_value="PTPM_CON", 
                     impute_function=impute, show_warnings=False, default_fc_parameters=settings.ComprehensiveFCParameters())

Feature Extraction: 100%|██████████| 160/160 [11:15<00:00,  4.22s/it] 


In [19]:
# Restablecer el índice
X = X.reset_index()

# Renombrar las columnas del índice
X.columns = ['CodigoEstacion', 'Fecha'] + list(X.columns[2:])

# Convertir la columna 'Fecha' al tipo de datos datetime 
X['Fecha'] = pd.to_datetime(X['Fecha'])

In [14]:
df['PTPM_CON_shifted'] = df.groupby('CodigoEstacion')['PTPM_CON'].shift(-1)
df.dropna(subset=['PTPM_CON_shifted'], inplace=True)

In [82]:
# Realizamos la combinación entre X_df y y_df usando las columnas 'Fecha' y 'CodigoEstacion'
model_df = pd.merge(X, df[['Fecha', 'CodigoEstacion', 'PTPM_CON_shifted']], 
                on=['Fecha', 'CodigoEstacion'], 
                how='left')
model_df.dropna(subset=['PTPM_CON_shifted'], inplace=True)

In [ ]:
# Diccionario para almacenar resultados de correlación por estación
resultados_correlacion = {}

# Iterar por cada estación única
for estacion in model_df['CodigoEstacion'].unique():
    # Filtrar datos por estación
    df_estacion = model_df[model_df['CodigoEstacion'] == estacion]
    
    # Seleccionar columnas para correlación
    columnas_correlacion = [col for col in df_estacion.columns 
                            if col not in ['CodigoEstacion', 'Fecha', 'PTPM_CON_shifted']]
    
    # Calcular correlaciones
    try:
        correlaciones = df_estacion[columnas_correlacion + ['PTPM_CON_shifted']].corr()['PTPM_CON_shifted']
        
        # Eliminar la correlación consigo mismo
        correlaciones = correlaciones.drop('PTPM_CON_shifted')

        # Ordenar correlaciones por valor absoluto
        correlaciones = correlaciones.abs().sort_values(ascending=False)

        # Almacenar resultados
        resultados_correlacion[estacion] = correlaciones
    
    except Exception as e:
        print(f"Error procesando estación {estacion}: {e}")

In [79]:
# Crear un DataFrame resumen de correlaciones
df_correlacion = pd.DataFrame(resultados_correlacion)
df_correlacion['Promedio'] = df_correlacion.mean(axis=1)
df_correlacion = df_correlacion.sort_values(by='Promedio', ascending=False)

# Extraemos las tres primeras palabras de cada característica
df_correlacion['Clase'] = df_correlacion.index.to_series().apply(lambda x: '__'.join(x.split('__')[:2]))

# Mantenemos las 30 características con mayor promedio de correlación que no sean del mismo tipo
df_top_30 = df_correlacion.drop_duplicates(subset='Clase').head(30)

# Mostramos el DataFrame resultante
print(df_top_30['Promedio'])

PTPM_CON__cwt_coefficients__coeff_6__w_5__widths_(2, 5, 10, 20)            0.407875
PTPM_CON__mean                                                             0.391354
PTPM_CON__fft_coefficient__attr_"abs"__coeff_0                             0.391354
PTPM_CON__sum_values                                                       0.391354
PTPM_CON__quantile__q_0.7                                                  0.372223
PTPM_CON__root_mean_square                                                 0.368809
PTPM_CON__median                                                           0.356659
PTPM_CON__abs_energy                                                       0.319528
PTPM_CON__absolute_maximum                                                 0.319499
PTPM_CON__maximum                                                          0.319499
PTPM_CON__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"    0.308709
PTPM_CON__range_count__max_1__min_-1                                       0

In [83]:
# Filtramos los índices donde 'Promedio' es mayor a 0.3 excluyendo 'PTPM_CON__absolute_maximum'
columnas = df_top_30[(df_top_30['Promedio'] > 0.3) & (df_top_30.index != 'PTPM_CON__absolute_maximum')].index.tolist()

# Extraemos únicamente esas columnas a model_df
model_df = model_df[['CodigoEstacion', 'Fecha', 'PTPM_CON_shifted'] + columnas]

# Guardamos el DataFrame resultante
model_df.to_csv(DATA_FOLDER + '/data_modelo.csv', index=False)